In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset

In [ ]:
from ray import tune

In [ ]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")

In [ ]:
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)
updated_df

In [ ]:
#Let's create an AutoLSTM model that is trained up to October 5, fix these parameters and create expanding window models
oct1stweekdf = updated_df.loc[:"2024-10-05"]
oct1stweekdf['ds'] = oct1stweekdf.index
oct1stweekdf["unique_id"] = "series_1"
oct1stweekdf = oct1stweekdf.rename(columns = {"value": "y"})
oct1stweekdf

In [ ]:
dataset, *_ = TimeSeriesDataset.from_df(oct1stweekdf)

In [ ]:
#Batch Size Exploration
config = dict(batch_size = tune.choice([8, 16, 32, 64]), encoder_hidden_size = 200, encoder_n_layers = 2, context_size = 10, decoder_hidden_size = 200, learning_rate = 0.001, max_steps = 1000, input_size = -1, inference_input_size = -1)
LSTMmodel = AutoLSTM(h = 4, backend = "ray", config = config)
LSTMmodel.fit(dataset = dataset)